## Regulation at the source? Comparing upstream and downstream climate policies

*An agent-based model to evaluate and compare the performance of upstream and downstream climate policies.*

Joël Foramitti, Ivan Savin, Jeroen C.J.M. van den Bergh

### 1. Initialization

In [ ]:
## Libraries ##

import numpy as np
import pandas as pd
import csv

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from SALib.sample import saltelli
from SALib.analyze import sobol

In [ ]:
## Import Model ##

from UvsD_ABM_Parameters import *
from UvsD_ABM_Agents import *
from UvsD_ABM_Model import *

In [ ]:
## Settings ##
open_economy = True 
do_multi_run = True
analyze_multi_run = False # Show evaluation criteria plot
analyze_sensitivity = False # Show sensitivity analysis plot
sensitivity_strength = 1000 # Number of runs: num_vars * 2 + 2
plot_sensitivity = True 
fs = 13 # Font size for plots

# Seperate simulation into smaller batches
batch_seperation = True
batch_total = 10
batch_current = 0 # start counting from 0!

In [ ]:
# Perform Multiple Runs

def multi_run(scenarios,param_range,open_economy=False):
    
    # Prepare sample
    param_values_multi = saltelli.sample(param_range, sensitivity_strength) 
    
    if batch_seperation:
        batch_len = int ( len(param_values_multi) / batch_total )
        param_values_multi = param_values_multi[batch_len*batch_current:batch_len*(batch_current+1)]
    
    measures = []
    n_err = 0
    n_err_s = 0
    err = False
    
    for i,pv in enumerate(param_values_multi): 
        
        if i == 0:
            print("Single run-time:")
            results = %time run_model(scenarios,param_values=pv,open_economy=open_economy)        
            print("\nScheduled runs: ",len(param_values_multi))
        else: 
            results = run_model(scenarios,param_values=pv,open_economy=open_economy)
            
        # Check for errors and try to repeat
        err = False
        for sc in results:
            if sc[1].error == True: 
                err = True
                      
        if err == True: 
            n_err +=1  
            c=0
            while True:
                results = run_model(scenarios,param_values=pv,open_economy=open_economy)
                
                err = False
                for sc in results:
                    if sc[1].error == True: 
                        err = True

                if err == False:
                    n_err_s +=1
                    break
                else: 
                    c+=1
                    if c>10: break 
                
        measures.extend( evaluation_measures(results,i) )
        
        print('\rDone: ' + str(i+1) + ' (' + str(n_err) + ' Errors, ' + str(n_err_s) + ' resolved)', end='')
      
    print("\n\nTotal run-time:")
    
    return measures

In [ ]:
# 111 Main Simulation over Parameter Range
if do_multi_run:
    measures = %time multi_run(scenarios,param_range,open_economy)

In [ ]:
measure_names = ['Run','Scenario',
                 'Emissions',
                 'Technology adoption','Compositional change','Production decline','Abatement, relative to target',
                 'Sales price',
                 'Upstream production costs','Downstream production costs','Upstream profits','Downstream profits','Policy revenue',
                 'Consumer Impact',
                 'Upstream Market Concentration','Downstream Market Concentration','Sales',
                 'Emission Costs','Emission & Fuel Costs',
                 'Local Emissions',
                 'Local Technology adoption','Local Compositional change','Local Production decline','Local Abatement, relative to target',
                 'Local Sales price',
                 'Local Upstream production costs','Local Downstream production costs','Local Upstream profits','Local Downstream profits','Local Policy revenue',
                 'Local Consumer Impact',
                 'Local Upstream Market Concentration','Local Downstream Market Concentration','Local Sales'
                ]

measure_names2 = ['Run','Scenario',
                 'Emissions',
                 'Technology \n adoption','Compositional \n change','Production \n decline','Abatement, relative to target',
                 'Sales \n price',
                 'Upstream \n production \n costs','Downstream \n production \n costs','Upstream \n profits','Downstream \n profits','Policy \n revenue',
                'Consumer \n Impact',
                 'Upstream \n Market \n Concentration','Downstream \n Market \n Concentration','Sales',
                  'Emission \n Costs','Emission & \n Fuel Costs',
                  'Local Emissions',
                 'Local Technology \n adoption','Local Compositional \n change','Local Production \n decline','Local Abatement, relative to target',
                 'Local Sales \n price',
                 'Local Upstream \n production \n costs','Local Downstream \n production \n costs','Local Upstream \n profits','Local Downstream \n profits','Local Policy \n revenue',
                 'Local \n Consumer \n Impact',
                 'Local Upstream \n Market \n Concentration','Local Downstream \n Market \n Concentration','Local Sales'
                 
                ]
if do_multi_run:
    
    # Create Pandas Dataframe
    df = pd.DataFrame( measures , columns = measure_names)
    
    if batch_seperation: 
        if open_economy:
            df.to_csv('Outputs/results_UVSP_open_batch_'+str(batch_current)+'.csv',index=False)
        else:
            df.to_csv('Outputs/results_UVSP_batch_'+str(batch_current)+'.csv',index=False)
        print('Saved batch', batch_current)
    else: 
        if open_economy:
            df.to_csv(r'Outputs/results_UVSP_open.csv',index=False)
        else:
            df.to_csv(r'Outputs/results_UVSP.csv',index=False)
        print('Saved normally')